In [38]:
import glob
import sys
sys.path.append('../../../msrv-xc')
import msrvxc

import numpy as np
import pickle

from astropy.io import fits
from astropy.table import table

from astropy.coordinates import SkyCoord
import astropy.units as u

In [37]:
with open('../../../msrv-xc/grids/interp_bosz.pkl', 'rb') as f:
    interp_bosz = pickle.load(f)
    
with open('../../../msrv-xc/grids/interp_bosz_norm.pkl', 'rb') as f:
    interp_bosz_norm = pickle.load(f)
    
wvl = np.load('../../../msrv-xc/grids/wavl.npy')

NameError: name 'pickle' is not defined

In [36]:
def continuum_normalize(wl, fl, ivar = None, avg_size = 150, ret_cont = False):
    
    fl_norm = np.zeros(np.size(fl))
    fl_cont = np.zeros(np.size(fl))
    
    ivar_yes = 0
    if ivar is not None:
        ivar_yes = 1
        ivar_norm = np.zeros(np.size(fl))
        
    for i in range(np.size(wl)):
        wl_clip = ((wl[i]-avg_size/2)<wl) * (wl<(wl[i]+avg_size/2))
        fl_cont[i] = np.median(fl[wl_clip])
        if ivar_yes:
            ivar_norm[i] = ivar[i]*np.median(fl[wl_clip])**2
    
    fl_norm = fl/fl_cont
    
    if ret_cont:
        if ivar_yes:
            return wl, fl_norm, ivar_norm, fl_cont
        else:
            return wl, fl_norm, fl_cont
    else:
        if ivar_yes:
            return wl, fl_norm, ivar_norm
        else:
            return wl, fl_norm

In [32]:
files = glob.glob('./*.fits')

In [33]:
f = fits.open(files[0])

In [34]:
loc = SkyCoord(f[0].header['RA'], f[0].header['DEC'], unit=(u.hour, u.degree))

In [35]:
ms_wl = f[0].header['CRVAL1'] + f[0].header['CD1_1'] * np.arange(f[0].data.shape[2])
ms_fl = f[0].data[0,0,:]